# Using APIs: Get DATA from FRED and World Bank

$\textbf{by Ahmed Pirzada, University of Bristol}$

$\textbf{aj.pirzada@bristol.ac.uk}$

$\textbf{27th October 2025}$

## Learning Objectives

- Import libraries for data and plotting (pandas, matplotlib).
- Fetch macroeconomic series from FRED using `fredapi` and inspect metadata.
- Combine series into a single DataFrame and compute simple statistics and correlations.
- Visualise time series using subplots with Matplotlib.
- Use World Bank `wbgapi` to explore sources and pull selected indicators.
- Clean/reshape data (indexes, labels) and compute summaries (e.g., group means).

## 1. IMPORT libraries

You will import core tools:
- `pandas` for tables and data ops.
- `matplotlib` for plotting.

In [ ]:
# Import Pandas for data tables and operations.
import pandas as pd
from matplotlib import pyplot as plt  # For plotting charts

## 2. Macro data from FRED

You will:
- Configure FRED access and fetch key series (GDP, unemployment, CPI).
- Inspect a series and combine multiple series into one DataFrame.
- Compute quick stats (e.g., correlations) and plot lines/subplots.

In [ ]:
# Install FRED client
!pip install fredapi

In [ ]:
# Configure FRED client to access economic time series.
from fredapi import Fred  # To access economic data from FRED
fred = Fred(api_key='...')  # Setup your FRED account and get you API key

Get data on CPI, GDP and Unemployment
- GDP: series name GDP
- Unemployment: series name UNRATE
- CPI: series name CPIAUCNS

In [ ]:
# Inspect metadata for a FRED series (title, units, frequency).
fred.get_series_info('GDP')

In [ ]:
# Download GDP time series from FRED.
gdp = fred.get_series('GDP', frequency='q', observation_start='1990-01-01')  # get quarterly GDP data
gdp.name = 'GDP'

In [ ]:
gdp.head()  # Display first few rows of GDP data

$\textbf{To-do:}$ Do the same for unemployment rate and CPI

In [ ]:
# Download unemployment rate series from FRED and give it a name.
unemp = ...

In [ ]:
# Display first few rows of unemployment rate data


In [ ]:
# Download CPI series from FRED and give it a name.
cpi = ...

In [ ]:
# Display first few rows of CPI data


---

$\textbf{Transform GDP and CPI data to growth rates}$

In [ ]:
gdp_growth = gdp.pct_change().dropna() * 100  # calculate GDP growth rate
gdp_growth.name = 'GDP Growth Rate'

In [ ]:
gdp_growth.head()  # Display first few rows of GDP growth rate

$\textbf{To-do:}$ Do the same for CPI

In [ ]:
# Calculate inflation rate from CPI data and give name.
inflation = ...

In [ ]:
# Display first few rows of inflation rate.


---

$\textbf{Form a DataFrame and Analyse}$

In [ ]:
# Combine series into one DataFrame aligned by date.
df_macro = pd.concat([gdp_growth, unemp, inflation], axis=1)

In [ ]:
# Preview the first few rows of the combined DataFrame.
df_macro.head()

$\textbf{To-do:}$ Drop missing observations

In [ ]:
# Remove rows with missing data


In [ ]:
# Display the first few rows of the final DataFrame.


In [ ]:
# Compute correlation matrix across macro series.


---

$\textbf{Visualisation}$

In [ ]:
# Plot all series in a single line chart.
plt.figure(figsize=(10, 6))
df_macro.plot()
plt.title('Macroeconomic Indicators Over Time')
plt.grid(True)
plt.show()

$\textbf{To-do:}$ Subplots

In [ ]:
# Plot each series in a subplot.


---

## 2. Data from World Bank

You will:
- Explore WDI sources/indicators with `wbgapi`.
- Pull selected indicators across years, tidy the index, and rename columns.
- Compute correlations and plot lines.
- Build a 2022 snapshot, merge regional/income metadata, and group by income.

In [ ]:
# Install World Bank API
!pip install wbgapi

In [ ]:
# Load World Bank API (wbgapi) to access WDI.
import wbgapi as wb

$\textbf{Get data from WDI database:}$
- GDP Growth: series name NY.GDP.MKTP.KD.ZG
- Inflation: series name FP.CPI.TOTL.ZG
- Unemployment: SL.UEM.TOTL.ZS

In [ ]:
# Info on a specific series.
wb.series.info('NY.GDP.MKTP.KD.ZG')

In [ ]:
# List of selected lindicators.
variables = ['NY.GDP.MKTP.KD.ZG', 'FP.CPI.TOTL.ZG', 'SL.UEM.TOTL.ZS']  # GDP growth, Inflation, Unemployment

$\textbf{Time series data}$

In [ ]:
# Country name
country = 'GBR'  # United Kingdom

In [ ]:
# Fetch selected indicators across years into a DataFrame.
df_ts = wb.data.DataFrame(variables, time=range(1990, 2025,1), economy=country, index='time', db=2)

In [ ]:
# Rename columns for clarity.
df_ts = df_ts.rename(columns={
    'NY.GDP.MKTP.KD.ZG': 'GDP Growth',
    'FP.CPI.TOTL.ZG': 'Inflation',
    'SL.UEM.TOTL.ZS': 'Unemployment'
})

$\textbf{To-do:}$ Clear and analyse data

In [ ]:
# Display the first few rows of the DataFrame.


In [ ]:
# Drop rows with missing data.


In [ ]:
# Calculate correlation matrix.


---

In [ ]:
# Clean year index and convert to integers.
df_ts.index = df_ts.index.str.replace('YR', '').astype(int)

$\textbf{Cross-section data}$

In [ ]:
# Choose a year for cross-sectional data
year = 2022

In [ ]:
# Fetch selected indicators across years into a DataFrame.
df_cs = wb.data.DataFrame(variables, time=year, index='economy', db=2)

In [ ]:
# Retrieve country metadata (region, income group).
meta = pd.DataFrame(wb.economy.list())
# Join data with metadata for grouped analysis.
df_cs = df_cs.reset_index().merge(meta[['id', 'region', 'incomeLevel']], left_on='economy', right_on='id', how='left')

In [ ]:
# Rename columns for clarity.
df_cs = df_cs.rename(columns={
    'NY.GDP.MKTP.KD.ZG': 'GDP Growth',
    'FP.CPI.TOTL.ZG': 'Inflation',
    'SL.UEM.TOTL.ZS': 'Unemployment',
    'economy': 'Economy',
    'incomeLevel': 'Income Grp',
    'region': 'Region'
})

In [ ]:
# Display the first few rows of the final DataFrame.
df_cs.head()

In [ ]:
# Drop id column.
df_cs.drop(columns=['id'], inplace=True)

$\textbf{To-do:}$ clean dataset

In [ ]:
# Drop rows with missing data.


In [ ]:
# Calculate correlation matrix for the three indicators.


In [ ]:
# Use groupby to compute the median for the three indicators by income group


---

$\textbf{Visualisation}$

In [ ]:
# Plot histogram for GDP Growth rates.
plt.figure(figsize=(8, 5))
plt.hist(df_cs['GDP Growth'].dropna(), bins=20, color='skyblue')
plt.title(f'Histogram of GDP Growth Rates in {year}')
plt.xlabel('GDP Growth Rate (%)')
plt.show()

$\textbf{To-do:}$ scatter plot between two variables

In [ ]:
# Plot a scatter plot between two variables


---

# Student Notes: Code Explanations

Use this recap to understand what each section is doing and how to read the outputs.

1) Import libraries
- `pandas` handles tabular data and operations.
- `matplotlib` plots charts.
- `fredapi` connects to the Federal Reserve Economic Data (FRED).
- `wbgapi` pulls World Bank indicators (WDI).

2) Macro data from FRED
- Create a FRED client and fetch time series like GDP, unemployment, and CPI.
- `get_series_info()` shows a series' metadata (title, units, frequency).
- Combine multiple series into one DataFrame with aligned dates (columns = series).
- Inspect with `head()`; compute `corr()` to compare co-movements.
- Plot series to see trends across time; use titles/labels to clarify.

3) Data from World Bank (WDI)
- Explore sources and indicator catalogue with `wb.source.info()` and `wb.series.list()`.
- Fetch selected indicators across years into a DataFrame and tidy the year index.
- Compute correlations and visualise time series.
- Build a single-year snapshot (e.g., 2022 inflation), then add metadata (region, income).
- Group by income (or region) to compute summary statistics (e.g., mean inflation).

Tips
- If an object is missing (NameError), re-run the earlier cell that creates it.
- Use `type(obj)`, `obj.shape`, and `obj.head()` to quickly check what you have.
- When APIs error, confirm keys/codes and your internet/API credentials.